# Image Preprocess

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
	sys.path.insert(0, str(PROJECT_ROOT))

import cv2 as cv
import numpy as np

from Code.image import ImgPreproc, ImgPreprocCfg

Imgcfg = ImgPreprocCfg(
	target_size=512,
	sigma=2.0,
	flag_refine_mask=True,
	open_ksize=5,
	close_ksize=5,
)
pre = ImgPreproc(cfg=Imgcfg)


run = 19
dataset = 6

# --- Configuración de paths ---
in_path = PROJECT_ROOT / "Database" / "data" / f"image{dataset}"

out_path = PROJECT_ROOT / "Database" / "tmp" / "image" / f"Preprocess Intento {run:02d}"
out_path.mkdir(parents=True, exist_ok=True)
labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]

for l in labels:
	input_dir = in_path / l
	out_dir = out_path / l
	out_dir.mkdir(parents=True, exist_ok=True)

paths = list(in_path.glob("*/*.jpg"))

for p in paths:
	i_bgr = cv.imread(str(p))
	if i_bgr is None:
		continue

	img_sq, mask_sq = pre.procesar(i_bgr, blacknwhite=False)

	out_dir = out_path / p.parent.name
	out_dir.mkdir(parents=True, exist_ok=True)

	cv.imwrite(str(out_dir / f"{p.stem} recortada.png"), img_sq)
	cv.imwrite(str(out_dir / f"{p.stem} mask.png"), mask_sq)




# Image Features

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
	sys.path.insert(0, str(PROJECT_ROOT))

import cv2 as cv
import numpy as np
import pandas as pd

from Code.image import ImgPreproc, ImgPreprocCfg
from Code.image import ImgFeat, hyper_params

Imgcfg = ImgPreprocCfg(
	target_size=512,
	sigma=2.0,
	flag_refine_mask=True,
	open_ksize=5,
	close_ksize=5,
)
pre = ImgPreproc(cfg=Imgcfg)

hp = hyper_params(
	radial_var_t_low=0.0,
	radial_var_t_high=0.045,
	r_hull_t=0.225,
)
feat = ImgFeat(hp=hp, mode="3D")

run = 3
dataset = 6

# --- Configuración de paths ---
in_path = PROJECT_ROOT / "Database" / "data" / f"image{dataset}"

out_path = PROJECT_ROOT / "Database" / "tmp" / "image" / f"Features Intento {run:02d}"
out_path.mkdir(parents=True, exist_ok=True)
labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]

rows = []
names = None
resultados = []
out_dir = []

for l in labels:
	out = out_path / l
	out.mkdir(parents=True, exist_ok=True)
	out_dir.append(out)

paths = sorted(in_path.glob("*/*.jpg"))

for p in paths:
	i_bgr = cv.imread(str(p))
	if i_bgr is None:
		continue

	img_sq, mask_sq = pre.procesar(i_bgr, blacknwhite=False)

	out = out_path / p.parent.name
	out.mkdir(parents=True, exist_ok=True)

	cv.imwrite(str(out / f"{p.stem} recortada.png"), img_sq)
	cv.imwrite(str(out / f"{p.stem} mask.png"), mask_sq)

	label = p.parent.name
	vec, names, debug = feat.extraer_features(img_sq, mask_sq)
	rows.append([label, p.name, *vec.tolist()])
	resultados.append(vec)

vec_alphas = [1]

X = np.asarray(resultados)
df_meta = pd.DataFrame(rows, columns=["clase", "archivo", *names])[["clase", "archivo"]]

for alpha in vec_alphas:
	X_scaled = X.copy()
	X_scaled[:, 2] *= alpha

	df_base = pd.DataFrame(X_scaled, columns=names)
	df_out = pd.concat([df_meta.reset_index(drop=True), df_base], axis=1)

	csv_path = out_path / f"features_alpha{alpha}.csv"
	df_out.to_csv(csv_path, index=False)
	display(df_out)




,clase,archivo,huecos,r hull,variacion radial
0,Arandela,Arandela01.jpg,1.0,0.266787,0.043520
1,Arandela,Arandela02.jpg,1.0,0.256724,0.049750
2,Arandela,Arandela03.jpg,1.0,0.247003,0.057159
3,Arandela,Arandela04.jpg,1.0,0.261023,0.075115
4,Arandela,Arandela05.jpg,1.0,0.265349,0.052414
5,Arandela,Arandela06.jpg,1.0,0.276186,0.049686
6,Arandela,Arandela07.jpg,1.0,0.276033,0.151717
7,Arandela,Arandela08.jpg,1.0,0.269334,0.091633
8,Arandela,Arandela09.jpg,1.0,0.270195,0.051264
9,Arandela,Arandela10.jpg,1.0,0.305843,0.056445


# K Means Model Tryout

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
	sys.path.insert(0, str(PROJECT_ROOT))

import cv2 as cv
import numpy as np
import pandas as pd

from Code.image import ImgPreproc, ImgPreprocCfg
from Code.image import ImgFeat, hyper_params
from Code.image import KMeansModel

Imgcfg = ImgPreprocCfg(
	target_size=512,
	sigma=2.0,
	flag_refine_mask=True,
	open_ksize=5,
	close_ksize=5,
)
pre = ImgPreproc(cfg=Imgcfg)

hp = hyper_params(
	radial_var_t_low=0.0,
	radial_var_t_high=0.045,
	r_hull_t=0.225,
)
feat = ImgFeat(hp=hp, mode="3D")

semilla_ = np.array([
	[1.0, 0.0, 0.0],    # Arandela
	[0.0, 0.0, 1.0],    # Clavo
	[0.0, 1.0, 1.0],    # Tornillo
	[1.0, 0.0, 1.0],    # Tuerca
], dtype=float)

model = KMeansModel(n_clusters=4, init_centers=semilla_)

run = 3
dataset = 6

# --- Configuración de paths ---
in_path = PROJECT_ROOT / "Database" / "data" / f"image{dataset}"

out_path = PROJECT_ROOT / "Database" / "tmp" / "image" / f"Model Intento {run:02d}"
out_path.mkdir(parents=True, exist_ok=True)
labels = ["Clavo", "Tornillo", "Tuerca", "Arandela"]

rows = []
names = None
resultados = []
out_dir = []

for l in labels:
	out = out_path / l
	out.mkdir(parents=True, exist_ok=True)
	out_dir.append(out)

paths = sorted(in_path.glob("*/*.jpg"))

for p in paths:
	i_bgr = cv.imread(str(p))
	if i_bgr is None:
		continue

	img_sq, mask_sq = pre.procesar(i_bgr, blacknwhite=False)

	out = out_path / p.parent.name
	out.mkdir(parents=True, exist_ok=True)

	cv.imwrite(str(out / f"{p.stem} recortada.png"), img_sq)
	cv.imwrite(str(out / f"{p.stem} mask.png"), mask_sq)

	label = p.parent.name
	vec, names, debug = feat.extraer_features(img_sq, mask_sq)
	rows.append([label, p.name, *vec.tolist()])
	resultados.append(vec)

vec_alphas = [1]

X = np.asarray(resultados)
df_meta = pd.DataFrame(rows, columns=["clase", "archivo", *names])[["clase", "archivo"]]

for alpha in vec_alphas:
	X_scaled = X.copy()
	X_scaled[:, 2] *= alpha

	df_base = pd.DataFrame(X_scaled, columns=names)
	df_out = pd.concat([df_meta.reset_index(drop=True), df_base], axis=1)

	csv_path = out_path / f"features_alpha{alpha}.csv"
	df_out.to_csv(csv_path, index=False)
	display(df_out)

model.fit(X)
centers = model._centers
labels = model.predict(X)
inertia = model._inertia

results = [{"seed": "manual", "inertia": inertia, "centers": centers, "labels": labels}]

df_runs = pd.DataFrame([{"seed": r["seed"], "inertia": r["inertia"]} for r in results])

rows = []
for r in results:
	for k, c in enumerate(r["centers"]):
		rows.append({"seed": r["seed"], "cluster": k, **{n: c[i] for i, n in enumerate(names)}})
df_centers = pd.DataFrame(rows)

display(df_runs)
display(df_centers)

,clase,archivo,huecos,r hull,variacion radial
0,Arandela,Arandela01.jpg,1.0,0.266787,0.043520
1,Arandela,Arandela02.jpg,1.0,0.256724,0.049750
2,Arandela,Arandela03.jpg,1.0,0.247003,0.057159
3,Arandela,Arandela04.jpg,1.0,0.261023,0.075115
4,Arandela,Arandela05.jpg,1.0,0.265349,0.052414
5,Arandela,Arandela06.jpg,1.0,0.276186,0.049686
6,Arandela,Arandela07.jpg,1.0,0.276033,0.151717
7,Arandela,Arandela08.jpg,1.0,0.269334,0.091633
8,Arandela,Arandela09.jpg,1.0,0.270195,0.051264
9,Arandela,Arandela10.jpg,1.0,0.305843,0.056445


,seed,inertia
0,manual,0.113985


,seed,cluster,huecos,r hull,variacion radial
0,manual,0,1.0,0.269448,0.067870
1,manual,1,0.0,0.299706,1.000000
2,manual,2,0.0,0.871420,1.000000
3,manual,3,1.0,0.260924,0.929623


# Orchestrator

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path().resolve().parents[1]
if str(PROJECT_ROOT) not in sys.path:
	sys.path.insert(0, str(PROJECT_ROOT))

import cv2 as cv
import numpy as np
import pandas as pd

from Code.image import ImgPreproc, ImgPreprocCfg
from Code.image import ImgFeat, hyper_params
from Code.image import KMeansModel
from Code.image import ImgOrchestrator
from Code.image import ImgOrchestrator

s0 = np.array([
	[1.0, 0.0, 0.0],    # Arandela
	[0.0, 0.0, 1.0],    # Clavo
	[0.0, 1.0, 1.0],    # Tornillo
	[1.0, 0.0, 1.0],    # Tuerca
], dtype=float)

orch = ImgOrchestrator()
informacion_entrenamiento = orch.entrenar(
	dataset_dir="Database/data/image6", 
	run=1, 
	seeds=s0, 
	output_root="Database/tmp/image/Orchestrator Model Intento 01",
	labels=["Clavo","Tornillo","Tuerca","Arandela"]
)

display(informacion_entrenamiento['df_runs'])
display(informacion_entrenamiento['df_centers'])

labels = ["Clavo","Tornillo","Tuerca","Arandela"]

out_root = informacion_entrenamiento["out_dir"]  # mismo que usó entrenar
out_root.mkdir(parents=True, exist_ok=True)

for n in range(1, 8):
    for l in labels:
        _, df = orch.predecir(f"Database/data/image{n}/{l}")
        df.to_csv(out_root / f"predicciones_image{n}_{l}.csv", index=False)




,seed,inertia
0,manual,0.113985


,seed,cluster,huecos,r hull,variacion radial
0,manual,0,1.0,0.269448,0.067870
1,manual,1,0.0,0.299706,1.000000
2,manual,2,0.0,0.871420,1.000000
3,manual,3,1.0,0.260924,0.929623
